# Spam Filter w/ Naive Bayes Theorem

The goal of this project is to develop a program that filters spam text messages. We will accomplish this by "teaching" our program how to classify messages as spam (or non-spam) using the multinomial Naive Bayes algorithm against a dataset of 5,000+ SMS messages previously classified by humans, found here.

In [26]:
import pandas as pd
import numpy as np

In [27]:
sms_spam = pd.read_csv('SMSSpamCollection',sep='\t',header=None, names=['Label','SMS'])
print('number of rows = {}'.format(sms_spam.shape[0]))
sms_spam.head(5)

number of rows = 5572


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [28]:
sms_spam['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

From a quick frequency table generation, we can see that 86.5% of the dataset is non-spam (ham), and 13.4% is spam. From a high level perspective, this makes sense, as text message spam is not very common.

## Creating Training and Test sets
To ensure we aren't introducing bias when we test if our program is running correctly, we need to develop a method of testing *before* we train our program. In this case, we will split the dataset into two groups:
- 80% in the training set
- 20% in the testing set

We picked these proportions as we want as large a training sample set as possible, while still having a test group large enough to test with.

We will be using the `DataFrame.sample()` method in order to sample our dataset.

In [29]:
#randomize dataset 
random_dataset = sms_spam.sample(frac=1, random_state=1)

#calculate index for split
training_test_index = round(len(random_dataset) * 0.8)

#Training/Test split
train_data = random_dataset[:training_test_index].reset_index(drop=True)
test_data = random_dataset[training_test_index:].reset_index(drop=True)

print(train_data.shape)
print(test_data.shape)

(4458, 2)
(1114, 2)


In [30]:
print('training set\n{}\n'.format(train_data['Label'].value_counts(normalize=True)))
print('testing set\n{}\n'.format(test_data['Label'].value_counts(normalize=True)))

training set
ham     0.86541
spam    0.13459
Name: Label, dtype: float64

testing set
ham     0.868043
spam    0.131957
Name: Label, dtype: float64



## Data Cleaning
In order to create an accurate vocabulary dictionary to use in our Naive-Bayes calculation, we first need to clean the SMS messages by normalizing the words within each message. By removing all punctuation and capitalization, we can ensure we are capturing, and later comparing, any matching words within our count-per-word vocabulary dictionary.

In [31]:
#before cleaning
train_data.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [32]:
#cleaning data
train_data['SMS'] = train_data['SMS'].str.replace('\W',' ')
train_data['SMS'] = train_data['SMS'].str.lower()
train_data.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [33]:
train_data['SMS'] = train_data['SMS'].str.split()

In [34]:
vocabulary = []
train_data['SMS'].head()
for message in train_data['SMS']:
    for word in message:
        vocabulary.append(word)
        
# vocabulary[:5]
vocabulary = list(set(vocabulary))

0                    [yep, by, the, pretty, sculpture]
1    [yes, princess, are, you, going, to, make, me,...
2                      [welp, apparently, he, retired]
3                                             [havent]
4    [i, forgot, 2, ask, ü, all, smth, there, s, a,...
Name: SMS, dtype: object

In [35]:
len(vocabulary)

0

In the cell above, we split each of the messages in our training set into a list of words. We then iterate through each message, adding every word to the list `vocabulary`. Finally, we condense `vocabulary` into a list containing a single instance of every word by first converting it into a set, and back to a list on the final line.

In [36]:
word_counts_per_sms = {unique_word: [0] * len(train_data['SMS']) for unique_word in vocabulary}
for index, sms in enumerate(train_data['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

KeyError: 'yep'

Here, we create a dictionary from the vocabulary we have built up, that we will use to count the number of times each word is used in . Each word is a key, and each value for that key is, initially, a list of zeros the same length as the total number of sms messages. As we iterate through each message again, every time a word is used in a message, the corresponding key-value list index for that message is added one to.  

In [ ]:
word_counts_frame = pd.DataFrame(word_counts_per_sms)
word_counts_frame.head()

In [ ]:
word_counts_frame['moan'].sum()

In [ ]:
training_set = pd.concat([train_data,word_counts_frame],axis=1,sort=False)
training_set.head()
training_set.shape[0]

## Finding Constants to use with Naive-Bayes Theorem.

Our equation to determine whether a SMS message is spam depends on comparing these two quantities:

$P(Spam|w_1 ,w_2 ,...,w_n ) \propto P(Spam) \cdot \prod_{k=1}^n   $
$P(Ham|w_1 ,w_2 ,...,w_n ) \propto P(Ham) \cdot \prod_{k=1}^n   $

whose elements consist of

$P(w_i | Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}$

$P(w_i | Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Hpam} + \alpha \cdot N_{Vocabulary}}$

In those elements, the following variables are constants, and can thus be found before our bulk calculations:
- P(Spam), P(Ham)
- $N_{spam}$, $N_{ham}$, $N_{vocabulary}$

In [ ]:
p_ham = (training_set['Label'] == 'ham').sum() / training_set.shape[0]
p_spam = (training_set['Label'] == 'spam').sum() / training_set.shape[0]
print('p_ham = {}'.format(p_ham))
print('p_spam = {}'.format(p_spam))

In [ ]:
n_ham = (training_set['Label'] == 'ham').sum()
n_spam = (training_set['Label'] == 'spam').sum()
n_vocab = (len(vocabulary)) # - 2 for 'Label' and 'SMS' columns
alpha = 1
print('n_vocab = {}'.format(n_vocab))
print('n_ham = {}'.format(n_ham))
print('n_spam = {}'.format(n_spam))

### Calculating probabilities for our vocabulary

In addition to the constants found above, each of the $P(w_i|Spam)$ terms, called *parameters* can also be calculated for each $w_i$ in the vocabularly


$P(w_i | Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}$


In [ ]:
p_w_spam = {unique_word: 0 for unique_word in vocabulary}
p_w_ham = {unique_word: 0 for unique_word in vocabulary}
training_spam = training_set[training_set['Label'] == 'spam']
training_ham = training_set[training_set['Label'] == 'ham']

In [ ]:

for word in vocabulary:
    n_w_spam = training_spam[word].sum()
    p_w_spam[word] = ((n_w_spam + alpha) /
                      (n_spam + alpha * n_vocab))
    
    n_w_ham = training_ham[word].sum()
    p_w_ham[word] = ((n_w_ham + alpha)/
                      (n_ham + alpha * n_vocab))    

In [ ]:
print(p_w_ham['fone'])
print(p_w_spam['fone'])

In [ ]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    
    for word in message:
        if word in p_w_spam:
            p_spam_given_message *= p_w_spam[word]
        if word in p_w_ham:
            p_ham_given_message *= p_w_ham[word]


    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [ ]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')
classify("Sounds good, Tom, then see u there")

In [ ]:
#edit above fcn to return values instead of print

def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    
    for word in message:
        if word in p_w_spam:
            p_spam_given_message *= p_w_spam[word]
        if word in p_w_ham:
            p_ham_given_message *= p_w_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'


In [ ]:
test_data['SMS'] = test_data['SMS'].str.replace('\W', ' ')
test_data['SMS'] = test_data['SMS'].str.lower()
test_data['predicted'] = test_data['SMS'].apply(classify_test_set)
test_data.head()

In [ ]:
correct = 0
total = test_data.shape[0]
for i in range(total):
    row = next(test_data.iterrows())[1]
    if row['Label'] == row['predicted']:
        correct += 1
accuracy = correct/total
print(accuracy)